In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
df=pd.read_csv("heart_clean_final.csv")
df2=df

In [4]:
indep_X=df2.drop('target',axis=1)
dep_Y=df2['target']

In [9]:
def rfeFeature(indep_X,dep_Y,n):
        rfelist=[]
        log_model = LogisticRegression(solver='lbfgs',max_iter=2000,random_state=42)
        RF = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
        DT= DecisionTreeClassifier(criterion = 'gini', max_features='sqrt',splitter='best',random_state = 42)
        svc_model = SVC(kernel = 'linear', random_state = 42)
        rfemodellist=[log_model,svc_model,RF,DT] 
        for i in   rfemodellist:
            print(i)
            log_rfe = RFE(estimator=i, n_features_to_select=n)
            log_fit = log_rfe.fit(indep_X,dep_Y)
            log_rfe_feature=log_fit.transform(indep_X)
            print("Selected features",indep_X.columns[log_fit.support_])
            rfelist.append(log_rfe_feature)
        return rfelist 

In [11]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        return X_train, X_test, y_train, y_test

In [13]:
def cm_prediction(classifier,X_test,y_test):
    y_pred = classifier.predict(X_test)
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    from sklearn.metrics import accuracy_score 
    from sklearn.metrics import classification_report 
    Accuracy=accuracy_score(y_test, y_pred )
    report=classification_report(y_test, y_pred)
    from sklearn.metrics import f1_score
    f1=f1_score(y_test,y_pred)
    return Accuracy,report,cm,f1

In [15]:
def logistic(X_train,y_train,X_test,y_test):      
    from sklearn.linear_model import LogisticRegression
    classifier = LogisticRegression(random_state = 0)
    classifier.fit(X_train, y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return  classifier,Accuracy,report,cm,f1

In [17]:
def svm_linear(X_train,y_train,X_test,y_test):
    from sklearn.svm import SVC
    classifier = SVC(kernel = 'linear', random_state = 0)
    classifier.fit(X_train, y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return  classifier,Accuracy,report,cm,f1

In [19]:
def svm_NL(X_train,y_train,X_test,y_test):
    from sklearn.svm import SVC
    classifier = SVC(kernel = 'rbf', random_state = 0)
    classifier.fit(X_train, y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return  classifier,Accuracy,report,cm,f1

In [21]:
def Naive(X_train,y_train,X_test,y_test):       
    from sklearn.naive_bayes import GaussianNB
    classifier = GaussianNB()
    classifier.fit(X_train, y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return  classifier,Accuracy,report,cm,f1     

In [23]:
def knn(X_train,y_train,X_test,y_test):
    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    classifier.fit(X_train, y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return  classifier,Accuracy,report,cm,f1

In [25]:
def Decision(X_train,y_train,X_test,y_test):
    from sklearn.tree import DecisionTreeClassifier
    classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return  classifier,Accuracy,report,cm,f1

In [27]:
def random(X_train,y_train,X_test,y_test):
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return classifier,Accuracy,report,cm,f1

In [29]:
def Xgboost(X_train,y_train,X_test,y_test):
    from xgboost import XGBClassifier
    classifier =XGBClassifier()
    classifier.fit(X_train, y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return classifier,Accuracy,report,cm,f1     

In [31]:
def AdaBoost(X_train,y_train,X_test,y_test):
    from sklearn.ensemble import AdaBoostClassifier
    classifier=AdaBoostClassifier()
    classifier.fit(X_train,y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return classifier,Accuracy,report,cm,f1 

In [33]:
def LightGBM(X_train,y_train,X_test,y_test):
    from lightgbm import LGBMClassifier
    classifier=LGBMClassifier(verbose=-1)
    classifier.fit(X_train,y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return classifier,Accuracy,report,cm,f1 
    

In [35]:
def rfe_classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf,accxg,accada,acclgbm): 
    rfedataframe=pd.DataFrame(index=['Logistic','SVC','Random','DecisionTree'],columns=['Logistic','SVMl','SVMnl','KNN','Naive','Decision','Random','XgBoost','AdaBoost','LightGBM'])
    for number,idex in enumerate(rfedataframe.index):
        rfedataframe.loc[idex,'Logistic']=acclog[number]       
        rfedataframe.loc[idex,'SVMl']=accsvml[number]
        rfedataframe.loc[idex,'SVMnl']=accsvmnl[number]
        rfedataframe.loc[idex,'KNN']=accknn[number]
        rfedataframe.loc[idex,'Naive']=accnav[number]
        rfedataframe.loc[idex,'Decision']=accdes[number]
        rfedataframe.loc[idex,'Random']=accrf[number]
        rfedataframe.loc[idex,'XgBoost']=accxg[number]
        rfedataframe.loc[idex,'AdaBoost']=accada[number]
        rfedataframe.loc[idex,'LightGBM']=acclgbm[number]
    return rfedataframe

In [37]:
X_train,X_test,y_train,y_test=split_scalar(indep_X,dep_Y)

In [63]:
rfelist=rfeFeature(indep_X,dep_Y,8)       
acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]
accxg=[]
accada=[]
acclgbm=[]
for i in rfelist:   
    X_train, X_test, y_train, y_test=split_scalar(i,dep_Y)   
    classifier,Accuracy,report,cm,f1=logistic(X_train,y_train,X_test,y_test)
    acclog.append(Accuracy)
    
    classifier,Accuracy,report,cm,f1=svm_linear(X_train,y_train,X_test,y_test)  
    accsvml.append(Accuracy)
    
    classifier,Accuracy,report,cm,f1=svm_NL(X_train,y_train,X_test,y_test)  
    accsvmnl.append(Accuracy)
    
    classifier,Accuracy,report,cm,f1=knn(X_train,y_train,X_test,y_test)  
    accknn.append(Accuracy)
    
    classifier,Accuracy,report,cm,f1=Naive(X_train,y_train,X_test,y_test)  
    accnav.append(Accuracy)
    
    classifier,Accuracy,report,cm,f1=Decision(X_train,y_train,X_test,y_test)  
    accdes.append(Accuracy)
    
    classifier,Accuracy,report,cm,f1=random(X_train,y_train,X_test,y_test)  
    accrf.append(Accuracy)

    classifier,Accuracy,report,cm,f1=Xgboost(X_train,y_train,X_test,y_test)  
    accxg.append(Accuracy)

    classifier,Accuracy,report,cm,f1=AdaBoost(X_train,y_train,X_test,y_test)  
    accada.append(Accuracy)

    classifier,Accuracy,report,cm,f1=LightGBM(X_train,y_train,X_test,y_test)  
    acclgbm.append(Accuracy)
    


LogisticRegression(max_iter=2000, random_state=42)
Selected features Index(['sex', 'cp', 'restecg', 'exang', 'oldpeak', 'slope', 'ca', 'thal'], dtype='object')
SVC(kernel='linear', random_state=42)
Selected features Index(['sex', 'cp', 'fbs', 'exang', 'oldpeak', 'slope', 'ca', 'thal'], dtype='object')
RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=42)
Selected features Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'thalach', 'oldpeak', 'ca'], dtype='object')
DecisionTreeClassifier(max_features='sqrt', random_state=42)
Selected features Index(['age', 'sex', 'cp', 'chol', 'thalach', 'oldpeak', 'slope', 'ca'], dtype='object')


In [65]:
result=rfe_classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf,accxg,accada,acclgbm)
highlighted_result=result.style.highlight_max(axis=1,color="lightgreen")
highlighted_result

,Logistic,SVMl,SVMnl,KNN,Naive,Decision,Random,XgBoost,AdaBoost,LightGBM
Logistic,0.869565,0.891304,0.891304,0.847826,0.869565,0.782609,0.804348,0.891304,0.847826,0.869565
SVC,0.869565,0.891304,0.891304,0.847826,0.869565,0.847826,0.826087,0.847826,0.847826,0.869565
Random,0.891304,0.913043,0.891304,0.891304,0.891304,0.826087,0.847826,0.869565,0.913043,0.869565
DecisionTree,0.869565,0.913043,0.869565,0.891304,0.891304,0.869565,0.847826,0.891304,0.869565,0.869565


In [67]:
rfelist=rfeFeature(indep_X,dep_Y,10)  
f1log=[]
f1svml=[]
f1svmnl=[]
f1knn=[]
f1nav=[]
f1des=[]
f1rf=[]
f1xg=[]
f1ada=[]
f1lgbm=[]
for i in rfelist:   
    X_train, X_test, y_train, y_test=split_scalar(i,dep_Y) 
    classifier,Accuracy,report,cm,f1=logistic(X_train,y_train,X_test,y_test)
    f1log.append(f1)
    
    classifier,Accuracy,report,cm,f1=svm_linear(X_train,y_train,X_test,y_test)  
    f1svml.append(f1)
        
    classifier,Accuracy,report,cm,f1=svm_NL(X_train,y_train,X_test,y_test)  
    f1svmnl.append(f1)
        
    classifier,Accuracy,report,cm,f1=knn(X_train,y_train,X_test,y_test)  
    f1knn.append(f1)
        
    classifier,Accuracy,report,cm,f1=Naive(X_train,y_train,X_test,y_test)  
    f1nav.append(f1)
        
    classifier,Accuracy,report,cm,f1=Decision(X_train,y_train,X_test,y_test)  
    f1des.append(f1)
        
    classifier,Accuracy,report,cm,f1=random(X_train,y_train,X_test,y_test)  
    f1rf.append(f1)
    classifier,Accuracy,report,cm,f1=Xgboost(X_train,y_train,X_test,y_test)  
    f1xg.append(f1)

    classifier,Accuracy,report,cm,f1=AdaBoost(X_train,y_train,X_test,y_test)  
    f1ada.append(f1)

    classifier,Accuracy,report,cm,f1=LightGBM(X_train,y_train,X_test,y_test)  
    f1lgbm.append(f1)
    


LogisticRegression(max_iter=2000, random_state=42)
Selected features Index(['age', 'sex', 'cp', 'fbs', 'restecg', 'exang', 'oldpeak', 'slope', 'ca',
       'thal'],
      dtype='object')
SVC(kernel='linear', random_state=42)
Selected features Index(['age', 'sex', 'cp', 'fbs', 'thalach', 'exang', 'oldpeak', 'slope', 'ca',
       'thal'],
      dtype='object')
RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=42)
Selected features Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'thalach', 'exang', 'oldpeak',
       'slope', 'ca'],
      dtype='object')
DecisionTreeClassifier(max_features='sqrt', random_state=42)
Selected features Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'thalach', 'exang', 'oldpeak',
       'slope', 'ca'],
      dtype='object')


In [69]:
def rfe_classification(f1log,f1svml,f1svmnl,f1knn,f1nav,f1des,f1rf,f1xg,f1ada,f1lgbm): 
    rfedataframe=pd.DataFrame(index=['Logistic','SVC','Random','DecisionTree'],columns=['Logistic','SVMl','SVMnl','KNN','Naive','Decision','Random','XgBoost','AdaBoost','LightGBM'])
    for number,idex in enumerate(rfedataframe.index):
        rfedataframe.loc[idex,'Logistic']=f1log[number]       
        rfedataframe.loc[idex,'SVMl']=f1svml[number]
        rfedataframe.loc[idex,'SVMnl']=f1svmnl[number]
        rfedataframe.loc[idex,'KNN']=f1knn[number]
        rfedataframe.loc[idex,'Naive']=f1nav[number]
        rfedataframe.loc[idex,'Decision']=f1des[number]
        rfedataframe.loc[idex,'Random']=f1rf[number]
        rfedataframe.loc[idex,'XgBoost']=f1xg[number]
        rfedataframe.loc[idex,'AdaBoost']=f1ada[number]
        rfedataframe.loc[idex,'LightGBM']=f1lgbm[number]
    return rfedataframe

In [71]:
result=rfe_classification(f1log,f1svml,f1svmnl,f1knn,f1nav,f1des,f1rf,f1xg,f1ada,f1lgbm)
highlighted_result=result.style.highlight_max(axis=1,color="lightgreen")
highlighted_result

,Logistic,SVMl,SVMnl,KNN,Naive,Decision,Random,XgBoost,AdaBoost,LightGBM
Logistic,0.933333,0.933333,0.931507,0.918919,0.916667,0.916667,0.916667,0.929577,0.935065,0.904110
SVC,0.945946,0.933333,0.945946,0.918919,0.944444,0.891892,0.944444,0.931507,0.935065,0.945946
Random,0.929577,0.945946,0.921053,0.921053,0.929577,0.882353,0.891892,0.921053,0.945946,0.921053
DecisionTree,0.929577,0.945946,0.921053,0.921053,0.929577,0.882353,0.891892,0.921053,0.945946,0.921053
